In [2]:
from steamwebapi.api import ISteamUser, IPlayerService, ISteamUserStats
import numpy as np
import pandas as pd
import json

STEAM_API_KEY = 'D518AC94ECCEE3B45FC0DCA2334329B9'

steamuserinfo = ISteamUser(steam_api_key=STEAM_API_KEY)
print(steamuserinfo)